In [7]:
import json
import pandas as pd
from datetime import datetime, timedelta, date

In [14]:
str(datetime.strptime('2023-10-31', "%Y-%m-%d").date())

'2023-10-31'

In [12]:
# import requests
# from requests.auth import HTTPBasicAuth

# def try_api():
  
#     # Your Jira instance URL
#     JIRA_URL = 'https://exalogicconsulting.atlassian.net'

#     # The API endpoint for issue search
#     API_ENDPOINT = '/rest/api/3/search'

#     # Your email and API token
#     EMAIL = 'shashank.raj@exalogic.co'
#     API_TOKEN = 'ATATT3xFfGF0ki1921cMCZooeOY60SYQ1rXo8VtUbzNKHGFyuZza9UV6ftMBuGPqchZD7BHGM82DydJTpTqejJQy1NjX4cxKZVsX_dWxv8YHhNiQCA37DmiHpkIL15Ulg13qmUK6r9PIplp6FI-3qVLvqdlUgVdVJnvJ5P15cSoa_FxIDCo7bis=77934F96'

#     # The JQL query
#     JQL_QUERY = 'project = "IN-13" AND status = "To Do"'

#     # The headers for the HTTP request
#     HEADERS = {
#         'Accept': 'application/json',
#     }

#     # The query parameters for the HTTP request
#     PARAMS = {
#         'jql': JQL_QUERY,
#     }

#     # Send a GET request to the Jira API
#     response = requests.get(
#         url=JIRA_URL + API_ENDPOINT,
#         headers=HEADERS,
#         params=PARAMS,
#         auth=HTTPBasicAuth(EMAIL, API_TOKEN)
#     )

#     # Print the JSON response from Jira
#     print(response.json())


In [13]:
def load_json_file(file_name):
  if not file_name.endswith(".json"):
    print("Only Josn file is expected. Quitting...")
    return
  else:
    try:
      f = open(file_name, encoding = 'utf8')
    except Exception:
      return Exception
    else:
      data = json.load(f)
      print("Loaded Json Data")
      return data

In [14]:
def transform_json_data(raw_json, df_act_emp):
  dict = {
  "user_names":[],
  "email_add":[],
  "jira_ids":[],
  "issue_key":[],
  "issue_id":[],
  "created":[],
  "started":[],
  "updated":[],
  "time_secs":[]
  }

  for x in raw_json:
    dict['user_names'].append(x["author"]["displayName"])
    dict['email_add'].append(x["author"]["emailAddress"])
    dict['jira_ids'].append(x["id"])
    dict['issue_key'].append(x["issue"]["key"])
    dict['issue_id'].append(x["issueId"])
    dict['time_secs'].append(x["timeSpentSeconds"])
    dict["created"].append(x["created"])
    dict["updated"].append(x["updated"])
    dict["started"].append(x["started"])
  
  df_jira = pd.DataFrame(dict)
  df_jira["created"] = pd.to_datetime(df_jira["created"])
  df_jira["started"] = pd.to_datetime(df_jira["started"])
  df_jira["updated"] = pd.to_datetime(df_jira["updated"])

  df_jira["created"] = df_jira["created"].dt.date
  df_jira["started"] = df_jira["started"].dt.date
  df_jira["updated"] = df_jira["updated"].dt.date

  df_merged_act_emp = df_act_emp.merge(
    df_jira,
    how='left',
    left_on="Email",
    right_on="email_add"
  )

  df_merged_act_emp["date_issue"] = str(datetime.today()-timedelta(days=int(0)+1)).split(" ")[0]
  # curr_dt = str(datetime.today()-timedelta(days=int(0)+1)).split(" ")[0]

  df_merged_act_emp["count"] = 1

  return df_merged_act_emp

In [15]:
location = "C:/Users/ShashankRaj/OneDrive - Exalogic Consulting/Documents/Proj Mgmt/Jira_Clockwork"

In [16]:
jira_data = load_json_file(location+"/Data_Files/my_log.json")

Loaded Json Data


In [17]:
df_act_emp = pd.read_csv(location+"/Data_Files/Act_Emp.csv")
final_df = transform_json_data(jira_data,df_act_emp)

In [18]:
final_df.to_csv(location+"/Data_Files/Jira_Data.csv", index=False)